# Primer to Prompt Engineering Lab

<center>
<img src="https://supportvectors.ai/logo-poster-transparent.png" width="400px" style="opacity:0.7">
</center>

In [1]:
%run supportvectors-common.ipynb


<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



## LLM Settings

- **Temperature**
  - Controls randomness of output.
  - **Low (e.g., 0.2)** → deterministic, factual.
  - **High (e.g., 0.8)** → creative, varied.
  - Best for creative tasks when high; factual tasks when low.

- **Top P (Nucleus Sampling)**
  - Limits selection to top tokens making up `p` probability mass.
  - **Low (e.g., 0.2)** → focused, confident answers.
  - **High (e.g., 0.9)** → broader, more diverse responses.
  - **Use either Temperature *or* Top P, not both.**

- **Max Length**
  - Sets the limit for generated tokens.
  - Controls verbosity, relevance, and cost.

- **Stop Sequences**
  - Defines strings that stop further generation.
  - Useful for structured output control (e.g., `"11"` to end a 10-item list).

- **Frequency Penalty**
  - Penalizes tokens based on **how often** they’ve appeared.
  - Higher value reduces **word repetition**.

- **Presence Penalty**
  - Penalizes if a token **has appeared before**, regardless of frequency.
  - Higher value encourages **topic diversity**; lower keeps model focused.

## Chat App vs. API

| Feature                     | Chat Application                            | API Usage                                           |
|----------------------------|---------------------------------------------|-----------------------------------------------------|
| **Prompt Format & Roles**  | Free-form input; implicit system behavior   | Structured messages with `system`, `user`, `assistant` |
| **Context Handling**       | Handled automatically                       | Requires full message history per request           |
| **Parameter Control**      | Limited or preset (e.g., sliders)           | Full access to `temperature`, `top_p`, etc.         |
| **Behavior Control**       | Basic tone/style via UI                     | Fine-tuned via system prompts and parameters         |
| **Workflow Automation**    | Manual only                                 | Scriptable pipelines, agents, batch jobs            |
| **Tool Integration**       | Limited to plugins or UI extensions         | Full function/tool calling and external tool use     |
| **Prompt Flexibility**     | Manual edits only                           | Dynamic prompts, templating, A/B testing             |
| **Output Control**         | Length managed internally                   | Customizable via `max_tokens`, `stop`, etc.          |
| **Streaming**              | Rare or limited                             | Fully supported (`stream=True`)                     |
| **Logging & Debugging**    | No access to usage data                     | Token usage, latency, cost details available         |
| **Provider Differences**   | Abstracted away                             | Must handle provider-specific formats                |

## Fundamentals of Prompting with LLM APIs

**The Core Concept: Message-Based Prompting**

At its core, prompting via API is about constructing a sequence of messages exchanged between a user and a model, optionally guided by a system instruction.

These messages form a dialogue — even for single-shot tasks.

Canonical Roles:
- system: sets behavior, tone, or constraints
- user: provides tasks, questions, or instructions
- assistant: stores previous responses (optional for history)
- (some APIs also support tools, function calls, or tool outputs)

**The Minimal Request Structure**

Every API interaction contains:
- A list of messages, usually as [ { role, content }, ... ]
- Model parameters, like temperature, top_p, max_tokens
- Optional settings like stop sequences, tool definitions, etc.

Example abstract structure:
```
{
  "model": "<model-name>",
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What's the capital of France?" }
  ],
  "temperature": 0.7,
  "top_p": 1.0,
  "max_tokens": 100,
  "stop": ["User:"]
}
```

**The Three Layers of Prompt Design**

a. Instruction Layer
    - Set expectations and behavior via system prompt or context.

b. Input Layer
    - Define user intent clearly — question, task, or data.

c. Output Framing Layer
    - Use examples, formatting hints, stop tokens, or few-shot samples to guide structure.

**Prompting is Functional Programming with Language**
- Each prompt is a pure function:

    ```output = LLM(prompt, params)```

- Context (messages) is your program state
- Parameters (temperature, etc.) are tuning knobs
- You can compose, refactor, debug, and version prompts like code

## OpenAI API

This introduces the core concepts behind the OpenAI API so you can start using it effectively for prompting, automation, and app development.

### What is the OpenAI API?

The OpenAI API allows developers to interact with language models like **GPT-4**, **GPT-3.5**, and **embedding models** via HTTPS requests. You send a structured request (usually JSON), and receive a generated response.

### Key Concepts

#### a. Endpoints

The main ones you’ll use:

| Endpoint           | Purpose                              |
|--------------------|--------------------------------------|
| `/v1/chat/completions` | Chat-based interaction with GPT-4 / GPT-3.5 |
| `/v1/completions`      | Legacy prompt-completion format   |
| `/v1/embeddings`       | Convert text into vector embeddings |


#### b. Message Structure

The chat endpoint uses a **message-based format**:

```json
  [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "Explain quantum computing in simple terms." }
  ]
```
##### A minimal `python` example calling the `OpenAI` API:
```python
from openai import OpenAI
api = OpenAI()
messages = [
  {"role": "system", "content": "You are a helpful assistant"},
  {"role": "user", "content": "Explain quantum computing in simple terms."}
  ]
response = api.chat.completions.create(model="gpt-4o", messages = messages)
```

**Roles** :
- system: Sets the model’s behavior or persona
- user: Represents the user’s input
- assistant: (Optional) previous model responses for context

**Parameters**:
- temperature
- top_p
- max_tokens
- stop
- stream

**API Key**
You need an API key from https://platform.openai.com/account/api-keys

## Instructor



**Instructor** is a lightweight Python library that helps you get **structured, typed, and validated outputs** from language models like OpenAI’s GPT or Anthropic’s Claude.

It works by combining:
- **LLMs (OpenAI, Claude, etc.)**
- **Pydantic** for defining strict data schemas
- A single function call to generate **safe, parseable** outputs

### Why Use Instructor?

LLMs are great at generating natural language, but:
- Their output is **unpredictable**
- You often need **structured formats** (JSON, dicts, typed objects)
- Manual parsing is error-prone

**Instructor solves this** by:
- Letting you define expected outputs as Python classes
- Handling prompting and validation under the hood
- Failing gracefully if the model output doesn't match the schema

```python
import instructor
from openai import OpenAI
from pydantic import BaseModel

client = instructor.from_openai(OpenAI())

class Person(BaseModel):
    name: str
    age: int

user_prompt = "Give me a person named Alice who is 30 years old"
result = self.api.chat.completions.create(model="gpt-4o", 
                                            messages=[
                                                        {"role": "user", "content": user_prompt}
                                                        ],
                                            response_model=Person,
                                            )
print(result.name)  # Alice
print(result.age)   # 30
```

**Supported LLMs**
- OpenAI (GPT-3.5, GPT-4, GPT-4o)
- Claude 3 (via Anthropic client)
- Mistral (via Together.ai, HuggingFace, etc.)
- Custom models (with plugin adapters)

**Use Cases**
- Extract structured data from unstructured text
- Turn user input into form-ready objects
- Power agents with typed memory/state
- Safe JSON generation from LLMs



## Libraries and Frameworks Used

#### **Core Dependencies:**
- **OpenAI** (`openai`) - OpenAI API client for GPT models
- **Instructor** (`instructor`) - Structured output framework for LLMs
- **Pydantic** (`pydantic`) - Data validation and model definition